In [1]:
!pip install -U efficientnet

     |████████████████████████████████| 50 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 35.8 MB 483 kB/s eta 0:00:01
     |████████████████████████████████| 178 kB 417 kB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 647 kB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 258 kB/s eta 0:00:01


In [3]:
import efficientnet.keras as efn
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(directory = "./train", batch_size = 20, class_mode = 'binary', target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory( directory = "./test", batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 18000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [5]:
from tensorflow.keras.applications import EfficientNetB0
model = EfficientNetB0(weights='imagenet')

2021-12-15 22:10:21.306076: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model = EfficientNetB0(include_top=False, weights='imagenet')

In [7]:
model = EfficientNetB0(weights='imagenet', drop_connect_rate=0.4)

In [8]:
for layer in model.layers:
    layer.trainable = False

In [9]:
from tensorflow.keras import Model 
from keras.layers import Dense, Activation, Flatten,Dropout

In [10]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(model.input,predictions)

In [13]:
model_final.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

mc = ModelCheckpoint(filepath = "./bestmodel.h5",
                     monitor = "accuracy", 
                     verbose = 1, 
                     save_best_only = True)
es = EarlyStopping(monitor = "accuracy",
                   min_delta = 0.01,
                   patience=5,
                   verbose=1)

cb = [mc,es]

In [17]:
eff_history = model_final.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch =100,epochs = 10,callbacks = cb)

Epoch 1/10
100/100 [==============================] - 36s 357ms/step - loss: 0.6935 - accuracy: 0.4800

Epoch 00001: accuracy improved from -inf to 0.48000, saving model to ./bestmodel.h5


/opt/homebrew/Caskroom/miniconda/base/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
100/100 [==============================] - 42s 423ms/step - loss: 0.6930 - accuracy: 0.5155

Epoch 00002: accuracy improved from 0.48000 to 0.51550, saving model to ./bestmodel.h5
Epoch 3/10
100/100 [==============================] - 44s 441ms/step - loss: 0.6932 - accuracy: 0.5130

Epoch 00003: accuracy did not improve from 0.51550
Epoch 4/10
100/100 [==============================] - 46s 454ms/step - loss: 0.6935 - accuracy: 0.4955

Epoch 00004: accuracy did not improve from 0.51550
Epoch 5/10
100/100 [==============================] - 45s 451ms/step - loss: 0.6937 - accuracy: 0.4875

Epoch 00005: accuracy did not improve from 0.51550
Epoch 6/10
100/100 [==============================] - 45s 451ms/step - loss: 0.6933 - accuracy: 0.4980

Epoch 00006: accuracy did not improve from 0.51550
Epoch 7/10
100/100 [==============================] - 46s 461ms/step - loss: 0.6933 - accuracy: 0.5010

Epoch 00007: accuracy did not improve from 0.51550
Epoch 00007: early stopping


In [25]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.inception_v3 import preprocess_input
import numpy as np

df = pd.read_csv('./test.csv')

path = df['path']

predictions = []
for paths in path:
    img = load_img(paths, target_size = (224, 224) )

    i = img_to_array(img)

    i = preprocess_input(i)

    input_arr = np.array([i])
    input_arr.shape

    pred = np.argmax(model_final.predict(input_arr))

    if pred == 0:
        predictions.append("fake")
    else:
        predictions.append("real")

In [34]:
path['label'] = predictions

In [35]:
path = df['path']

In [36]:
dff = pd.DataFrame(path)
dff['label'] = predictions
dff.to_csv('submit.csv', index = False)

In [45]:
img = load_img("data/test/00010.jpg", target_size = (224, 224))

i = img_to_array(img)

i = preprocess_input(i)

input_arr = np.array([i])
pred = np.argmax(model_final.predict(input_arr))

if pred == 0:
    print("fake")
else:
    print("real")
    


fake
